* Ray RLlib 노트북

필요 패키지 삽입

In [1]:
import gymnasium as gym
import numpy as np
import pandas as pd

from horcrux_terrain_v1.envs import SandWorld

import ray
from ray.rllib.algorithms import ppo
from ray.rllib.algorithms.ppo import PPOConfig

from ray.tune.registry import register_env


c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


Ray 실행 (Warning 관련 무시 키워드)

In [2]:
ray.init(runtime_env={"env_vars": {"PYTHONWARNINGS": "ignore::DeprecationWarning"}})

2024-09-19 15:04:21,062	INFO worker.py:1777 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.12.5
Ray version:,2.36.0
Dashboard:,http://127.0.0.1:8265


Gym -> Rllib Env 등록

In [3]:
env_config = {
    "forward_reward_weight": 2,
    "side_cost_weight": 0.2,
    "unhealthy_max_steps": 75,
    "healthy_roll_range": (-70,70)
}

register_env("sand-v1", lambda config: SandWorld(forward_reward_weight=env_config["forward_reward_weight"], 
                                                 side_cost_weight=env_config["side_cost_weight"], 
                                                 unhealthy_max_steps=env_config["unhealthy_max_steps"], 
                                                 healthy_roll_range=env_config["healthy_roll_range"]))

학습 알고리즘 설정

In [4]:
config = PPOConfig()
# Activate new API stack.
config.api_stack(
    enable_rl_module_and_learner=True,
    enable_env_runner_and_connector_v2=True,
)
config.environment("sand-v1")
config.framework("torch")
config.resources(num_gpus=1, num_cpus_for_main_process=12)
config.env_runners(num_env_runners=12)
config.training(
    gamma=0.9, 
    lr=0.0001, 
    # kl_coeff=0.3, 
    train_batch_size_per_learner=96000,              
    sgd_minibatch_size=32768, 
)
config.evaluation(evaluation_interval=100)

# See model catalog for more options.
# https://docs.ray.io/en/latest/rllib/rllib-models.html
# config.model["fcnet_hiddens"] = [512, 512, 512, 512, 512]
config.model["fcnet_hiddens"] = [1024, 1024, 1024, 1024, 1024]
config.model["use_lstm"] = True
# config.model["lstm_cell_size"] = 2048
config.model["lstm_cell_size"] = 4096

algo = config.build()


c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\ray\rllib\algorithms\algorithm.py:555: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may be remove

학습 시작

In [5]:
from pprint import pprint

n_iter = 150
save_iter = 0
save_name = "1024x5_4096_inf"

for i in range(n_iter):
    result = algo.train()
    print(f"{i:03d}th iteration done")
    result.pop("config")
    pprint(result)

    if i%60 == 0:
        checkpoint_dir = algo.save(save_name+"_"+str(save_iter))
        print(f"Checkpoint saved in directory {checkpoint_dir}")
        save_iter += 1

algo.save(save_name+str("_final"))

000th iteration done
{'date': '2024-09-19_15-05-28',
 'done': False,
 'env_runners': {'agent_episode_returns_mean': {'default_agent': -52.90069175846086},
                 'episode_duration_sec_mean': 11.674608151999855,
                 'episode_len_max': 6000,
                 'episode_len_mean': 2901.88,
                 'episode_len_min': 48,
                 'episode_return_max': 65.9587345725487,
                 'episode_return_mean': -52.90069175846086,
                 'episode_return_min': -197.32899528926217,
                 'module_episode_returns_mean': {'default_policy': -52.90069175846086},
                 'num_agent_steps_sampled': {'default_agent': 96000},
                 'num_agent_steps_sampled_lifetime': {'default_agent': 96000},
                 'num_env_steps_sampled': 96000,
                 'num_env_steps_sampled_lifetime': 96000,
                 'num_episodes': 25,
                 'num_module_steps_sampled': {'default_policy': 96000},
                 'num

(SingleAgentEnvRunner pid=31684) c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.num_envs to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.num_envs` for environment variables or `env.get_wrapper_attr('num_envs')` that will search the reminding wrappers.
(SingleAgentEnvRunner pid=31684)   logger.warn(
(SingleAgentEnvRunner pid=28776) c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.single_observation_space to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.single_observation_space` for environment variables or `env.get_wrapper_attr('single_observation_space')` that will search the reminding wrappers.
(SingleAgentEnvRunner pid=28776) c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.sin

001th iteration done
{'date': '2024-09-19_15-06-16',
 'done': False,
 'env_runners': {'agent_episode_returns_mean': {'default_agent': -65.37188386856714},
                 'episode_duration_sec_mean': 12.328801924675266,
                 'episode_len_max': 6000,
                 'episode_len_mean': 3064.740259740259,
                 'episode_len_min': 48,
                 'episode_return_max': 65.9587345725487,
                 'episode_return_mean': -65.37188386856714,
                 'episode_return_min': -200.01102346582644,
                 'module_episode_returns_mean': {'default_policy': -65.37188386856714},
                 'num_agent_steps_sampled': {'default_agent': 96000},
                 'num_agent_steps_sampled_lifetime': {'default_agent': 288000},
                 'num_env_steps_sampled': 96000,
                 'num_env_steps_sampled_lifetime': 1344000,
                 'num_episodes': 27,
                 'num_module_steps_sampled': {'default_policy': 96000},
        

(SingleAgentEnvRunner pid=31684) c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.num_envs to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.num_envs` for environment variables or `env.get_wrapper_attr('num_envs')` that will search the reminding wrappers.
(SingleAgentEnvRunner pid=31684)   logger.warn( [repeated 3x across cluster]
(SingleAgentEnvRunner pid=11624) c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.single_observation_space to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.single_observation_space` for environment variables or `env.get_wrapper_attr('single_observation_space')` that will search the reminding wrappers.
(SingleAgentEnvRunner pid=11624) c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\gymnasium\core.py:31

002th iteration done
{'date': '2024-09-19_15-07-04',
 'done': False,
 'env_runners': {'agent_episode_returns_mean': {'default_agent': -75.53948949272461},
                 'episode_duration_sec_mean': 12.934837046749935,
                 'episode_len_max': 6000,
                 'episode_len_mean': 3211.2375000000006,
                 'episode_len_min': 48,
                 'episode_return_max': 65.9587345725487,
                 'episode_return_mean': -75.53948949272461,
                 'episode_return_min': -221.70825690812939,
                 'module_episode_returns_mean': {'default_policy': -75.53948949272461},
                 'num_agent_steps_sampled': {'default_agent': 96000},
                 'num_agent_steps_sampled_lifetime': {'default_agent': 576000},
                 'num_env_steps_sampled': 96000,
                 'num_env_steps_sampled_lifetime': 3744000,
                 'num_episodes': 19,
                 'num_module_steps_sampled': {'default_policy': 96000},
       

...

(SingleAgentEnvRunner pid=31684) c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.num_envs to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.num_envs` for environment variables or `env.get_wrapper_attr('num_envs')` that will search the reminding wrappers.
(SingleAgentEnvRunner pid=28492) c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.single_observation_space to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.single_observation_space` for environment variables or `env.get_wrapper_attr('single_observation_space')` that will search the reminding wrappers. [repeated 4x across cluster]
(SingleAgentEnvRunner pid=28492) c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.single_action_space to

148th iteration done
{'date': '2024-09-19_17-18-28',
 'done': False,
 'env_runners': {'agent_episode_returns_mean': {'default_agent': -63.35681493430901},
                 'episode_duration_sec_mean': 18.035111120333326,
                 'episode_len_max': 6000,
                 'episode_len_mean': 3685.486666666666,
                 'episode_len_min': 43,
                 'episode_return_max': 82.61912818892651,
                 'episode_return_mean': -63.35681493430901,
                 'episode_return_min': -263.6207475447836,
                 'module_episode_returns_mean': {'default_policy': -63.35681493430901},
                 'num_agent_steps_sampled': {'default_agent': 96000},
                 'num_agent_steps_sampled_lifetime': {'default_agent': 1072800000},
                 'num_env_steps_sampled': 96000,
                 'num_env_steps_sampled_lifetime': -168645888,
                 'num_episodes': 26,
                 'num_module_steps_sampled': {'default_policy': 96000},
 

(SingleAgentEnvRunner pid=31684) c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.num_envs to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.num_envs` for environment variables or `env.get_wrapper_attr('num_envs')` that will search the reminding wrappers.
(SingleAgentEnvRunner pid=31684)   logger.warn( [repeated 3x across cluster]
(SingleAgentEnvRunner pid=32008) c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.single_observation_space to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.single_observation_space` for environment variables or `env.get_wrapper_attr('single_observation_space')` that will search the reminding wrappers.
(SingleAgentEnvRunner pid=32008) c:\Users\doore\anaconda3\envs\gdtor\Lib\site-packages\gymnasium\core.py:31

149th iteration done
{'date': '2024-09-19_17-19-23',
 'done': False,
 'env_runners': {'agent_episode_returns_mean': {'default_agent': -67.2327838557505},
                 'episode_duration_sec_mean': 19.173865394333234,
                 'episode_len_max': 6000,
                 'episode_len_mean': 3918.190000000001,
                 'episode_len_min': 67,
                 'episode_return_max': 120.93798436966568,
                 'episode_return_mean': -67.2327838557505,
                 'episode_return_min': -263.6207475447836,
                 'module_episode_returns_mean': {'default_policy': -67.2327838557505},
                 'num_agent_steps_sampled': {'default_agent': 96000},
                 'num_agent_steps_sampled_lifetime': {'default_agent': 1087200000},
                 'num_env_steps_sampled': 96000,
                 'num_env_steps_sampled_lifetime': 3098112,
                 'num_episodes': 28,
                 'num_module_steps_sampled': {'default_policy': 96000},
      

TrainingResult(checkpoint=Checkpoint(filesystem=local, path=1024x5_4096_inf_final), metrics={'timers': {'env_runner_sampling_timer': 41.305932836810854, 'learner_update_timer': 10.942161938752935, 'synch_weights': 0.02138478138362463, 'synch_env_connectors': 0.020592408505395475, 'training_iteration_time_sec': 55.39287440776825, 'restore_workers_time_sec': 0.0, 'training_step_time_sec': 55.39278004169464, 'restore_eval_workers_time_sec': 0.0, 'evaluation_iteration_time_sec': 107.80814599990845, 'evaluation_iteration_throughput': 0.0, 'synch_eval_env_connectors_time_sec': 0.0}, 'env_runners': {'episode_duration_sec_mean': 19.173865394333234, 'episode_len_mean': 3918.190000000001, 'num_agent_steps_sampled': {'default_agent': 96000}, 'episode_len_max': 6000, 'episode_return_mean': -67.2327838557505, 'num_env_steps_sampled': 96000, 'module_episode_returns_mean': {'default_policy': -67.2327838557505}, 'num_module_steps_sampled': {'default_policy': 96000}, 'sample': 36.352245092080146, 'agen

환경에서 학습된 Policy 테스트하기

In [6]:
from ray.rllib.core.rl_module import RLModule
import pathlib
import torch
import numpy as np
import gymnasium as gym
from horcrux_terrain_v1.envs import SandWorld

rl_module = RLModule.from_checkpoint(
    pathlib.Path("./1024x5_4096_inf_final") / "learner_group" / "learner" / "rl_module"
)["default_policy"]

In [8]:
env = gym.make("horcrux_terrain_v1/sand-v1", 
               terminate_when_unhealthy = False, 
               render_mode = "human", 
            #    render_camera_name = 'ceiling', 
               use_gait = True,
               gait_params = (30,30,40,40,0),
               **env_config,
               ) 


episode_return = 0
terminated = truncated = False

obs, info = env.reset()

torch_obs_batch = torch.from_numpy(np.array([obs]))

for i in range(6000):
   torch_obs_batch = torch.from_numpy(np.array([obs]))
   action_logits = rl_module.forward_inference({"obs": torch_obs_batch})[
      "action_dist_inputs"
    ]
   
   action = torch.argmax(action_logits[0]).numpy()
   obs, reward, terminated, truncated, info = env.step(action)

   episode_return += reward

print(f"Reached episode return of {episode_return}.")

env.close()

Reached episode return of -218.53879442435755.
